In [9]:
import os
import cv2
import numpy as np
import torch
from tqdm import tqdm

categories = {'no_fire': 0, 'fire': 1}

IMG_SIZE = 224
VIDEO_DIR = 'videos/'

def extract_frames_from_video(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frames.append(frame_resized)
    cap.release()
    return frames

def create_training_data():
    training_data = []
    for category in os.listdir(VIDEO_DIR):
        if category.startswith('.'):
            continue
        print(category)
        category_path = os.path.join(VIDEO_DIR, category)
        if os.path.isdir(category_path):
            for video_file in tqdm(os.listdir(category_path)):
                if video_file.endswith('.mp4'):
                    video_path = os.path.join(category_path, video_file)
                    frames = extract_frames_from_video(video_path)
                    for frame in frames:
                        frame_tensor = torch.from_numpy(frame.transpose((2, 0, 1))).float()
                        training_data.append([frame_tensor, categories[category]])
    return training_data


In [10]:
training_data = create_training_data()

fire


100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


no_fire


100%|██████████| 1/1 [00:24<00:00, 24.56s/it]


In [11]:
X = torch.stack([sample[0] for sample in training_data])
y = torch.Tensor([label for _, label in training_data])

In [12]:
X_train, X_test = torch.split(X, int(0.8 * len(X)), dim=0)
y_train, y_test = torch.split(y, int(0.8 * len(y)), dim=0)

In [13]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.apply_proj = stride != 1 or inplanes != planes
        if self.apply_proj:
            self.proj = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)
            self.proj_bn = nn.BatchNorm2d(planes)

    def forward(self, X):
        f = self.bn2(self.conv2(F.relu(self.bn1(self.conv1(X)))))

        if self.apply_proj:
            g = self.proj_bn(self.proj(X))
        else:
            g = X
        return F.relu(f + g)

class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.resnet = nn.Sequential(
            ResidualBlock(3, 16, stride=2),
            ResidualBlock(16, 32, stride=2),
            ResidualBlock(32, 64, stride=2)
        )
        self.fc1 = nn.Linear(64 * 28 * 28, 1000)
        self.fc2 = nn.Linear(1000, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.sigmoid(x)

model = BinaryClassifier()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))  # Unsqueeze per adattare la forma delle etichette
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs >= 0.5).squeeze().float()  # Considera 1 se l'output è >= 0.5, altrimenti 0
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f"Test Accuracy: {accuracy}")

Epoch [1/10], Loss: 10.890434834361077
Epoch [2/10], Loss: 11.40625
Epoch [3/10], Loss: 11.40625
Epoch [4/10], Loss: 11.40625
Epoch [5/10], Loss: 11.40625
Epoch [6/10], Loss: 11.40625
Epoch [7/10], Loss: 11.40625
Epoch [8/10], Loss: 11.40625
Epoch [9/10], Loss: 11.40625
Epoch [10/10], Loss: 11.40625
Test Accuracy: 1.0


In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x, dim=1)

model = CNN()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.argmax(-1).float(), labels)  # Convert labels to float and unsqueeze
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs >= 0.5).squeeze().float()  # Consider 1 if the output is >= 0.5, else 0
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f"Test Accuracy: {accuracy}")


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [73]:
import numpy as np
import tensorflow as tf

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
y = np.array(y)

In [75]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X, y, batch_size=32, epochs=3,validation_split=0.3)

Epoch 1/3
35/35 [==============================] - 68s 2s/step - loss: 0.1611 - accuracy: 0.9679 - val_loss: 4.4363e-08 - val_accuracy: 1.0000
Epoch 2/3
35/35 [==============================] - 66s 2s/step - loss: 3.3152e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/3
35/35 [==============================] - 67s 2s/step - loss: 4.0105e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [78]:
img = cv2.imread('videos/test.jpg')
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img.astype('float32') / 255.0

fire = tf.convert_to_tensor(img[np.newaxis, ...])

predictions = model.predict(fire)
np.argmax(predictions)

1/1 [==============================] - 0s 49ms/step


1

In [80]:
img = cv2.imread('videos/test2.jpg')
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img.astype('float32') / 255.0

non_fire = tf.convert_to_tensor(img[np.newaxis, ...])

predictions = model.predict(non_fire)
np.argmax(predictions)

1/1 [==============================] - 0s 64ms/step


1

In [ ]:
model2 = tf.keras.models.load_model('TrainedModels/Fire-64x64-color-v7-soft.h5')